In [82]:
import os, json, time, logging
from datetime import datetime, timedelta
import numpy as np
from scipy.optimize import differential_evolution
from scipy.stats.qmc import Halton

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit_aer.primitives import Sampler as AerSampler
#from qiskit_ibm_runtime import Estimator
from qiskit.quantum_info import SparsePauliOp

from susy_qm import calculate_Hamiltonian2

In [93]:
def measure_hermitian_expectation(H, state_circuit, shots=1024):
    # Diagonalize H
    # After eigendecomposition
    eigvals, eigvecs = np.linalg.eigh(H)

    # Sort eigenvectors based on projection onto computational basis states
    # This enforces a direct bitstring ↔ eigenvalue match
    bit_order = np.argsort([np.argmax(np.abs(vec)) for vec in eigvecs.T])
    eigvals = eigvals[bit_order]
    eigvecs = eigvecs[:, bit_order]

    U = eigvecs.T.conj()  # Eigenbasis rotation

    num_qubits = int(np.log2(H.shape[0]))

    # Create measurement circuit in eigenbasis of H
    meas_circuit = QuantumCircuit(num_qubits, num_qubits)
    meas_circuit.unitary(U.conj().T, range(num_qubits))  # Apply U†
    meas_circuit.measure(range(num_qubits), range(num_qubits))

    # Compose with state preparation
    full_circuit = state_circuit.compose(meas_circuit)

    # Sample from circuit
    sampler = AerSampler(run_options={"shots": shots})
    job = sampler.run([full_circuit])
    quasi_dist = job.result().quasi_dists[0]

    # Compute expectation value from sampled bitstrings
    expval = 0.0
    for bit, prob in quasi_dist.items():
        index = int(bit) if isinstance(bit, int) else int(bit, 2)
        expval += eigvals[index] * prob

    return expval.real

In [94]:
# Example 1-qubit Hermitian operator
H = np.array([[1, 1j], [-1j, 1]])

# Example circuit: Ry(π/4)
theta = np.pi / 4
qc = QuantumCircuit(1)
qc.ry(theta, 0)

# Compute expectation value of H using 1024 shots
expval = measure_hermitian_expectation(H, qc, shots=1024)
print("⟨H⟩ (sampled):", expval)


⟨H⟩ (sampled): 1.6953125


In [126]:
potential = 'AHO'
cutoff = 4
H = calculate_Hamiltonian2(cutoff, potential)
eigenvalues = np.sort(np.linalg.eig(H)[0])[:4]
min_eigenvalue = np.min(eigenvalues)

num_qubits = int(1 + np.log2(cutoff))
observable = SparsePauliOp.from_operator(H)

In [117]:
from qiskit.circuit.library import TwoLocal

num_qubits = 3  # because H is 8×8
qc = TwoLocal(
    num_qubits=num_qubits,
    rotation_blocks='ry',
    entanglement_blocks='cz',
    reps=2,
    parameter_prefix='θ'
)
param_objs = qc.parameters
num_params = len(param_objs)


In [127]:
num_params = 3
param_objs = [Parameter(f"θ{i}") for i in range(num_params)]

qc = QuantumCircuit(num_qubits)
#qc.x(0)
qc.ry(param_objs[0], 0)
qc.ry(param_objs[1], 1)
qc.ry(param_objs[2], 2)
#qc.ry(param_objs[3], 0)

In [128]:
def cost_fn(params):
    param_dict = dict(zip(param_objs, params))
    bound_circuit = qc.assign_parameters(param_dict, inplace=False)
    
    estimates = [
        measure_hermitian_expectation(H, bound_circuit, shots=1024)
        for _ in range(3)  # smoothing
    ]
    return np.mean(estimates)

In [129]:
bounds = [(0, 2 * np.pi) for _ in range(num_params)]
max_iter = 10000
strategy = "randtobest1bin"
tol = 1e-3
abs_tol = 1e-3
popsize = 20

seed = (os.getpid() * int(time.time())) % 123456789
run_start = datetime.now()

halton_sampler = Halton(d=num_params, seed=seed)
scaled_samples = 2 * np.pi * halton_sampler.random(n=popsize)


result = differential_evolution(
    cost_fn,
    bounds=bounds,
    maxiter=max_iter,
    tol=tol,
    atol=abs_tol,
    strategy=strategy,
    popsize=popsize,
    init=scaled_samples,
    seed=seed
    )

KeyboardInterrupt: 

In [124]:
# Output
print("\nVQE Result using Differential Evolution:")
print(f"  Minimum eigenvalue: {result.fun}")
print(f"  Optimal parameters: {result.x}")



VQE Result using Differential Evolution:
  Minimum eigenvalue: -0.4374999999999998
  Optimal parameters: [3.19343931 5.13988777]
